In [0]:
import time
import os
import os.path
from osgeo import gdal, ogr, osr
from scipy import ndimage
from scipy import misc
from io import StringIO
gdal.UseExceptions()
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline
import urllib
import pandas as pd
import numpy as np 

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
from keras.applications.vgg16 import VGG16
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.optimizers import SGD
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Dropout
from multiprocessing import Pool
import os
import time
import pandas as pd
import numpy as np
from keras.models import Model
import pickle 


Using TensorFlow backend.


In [4]:
#Mount GDrive to access satellite images data
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
#load images file names dictionary
with open('/content/drive/My Drive/Thesis/daytime_analysis/images_name_dict', 'rb') as fp:
    images_name_dict = pickle.load(fp)

OSError: ignored

In [0]:
#Make directory for unloading satellite images
os.mkdir('/content/google_images')

In [7]:
#Copy satellite files from Drive and unload into local directory of Colab
#os.chdir('/content/drive/My Drive/Thesis/')
!cp /content/drive/My\ Drive/Thesis/google_images.zip /content/google_images

cp: error reading '/content/drive/My Drive/Thesis/google_images.zip': Operation canceled


In [0]:
#Copy satellite files from Drive and unload into local directory of Colab
os.chdir('/content/google_images') 

cp: error reading '/content/drive/My Drive/Thesis/google_images.zip': Operation canceled


In [0]:
!unzip -qq google_images.zip

[google_images.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of google_images.zip or
        google_images.zip.zip, and cannot find google_images.zip.ZIP, period.


In [0]:
#Move satellite images data to folders according to their lightness intensity

import sys, string, os, shutil

def move_to_group(lightness_small, lightness_big, class_id):
    new_directory = 'data/google_image_cnn/class_' + str(class_id) + '/'
    if not os.path.isdir(new_directory):
        os.makedirs(new_directory)
    for i in range(lightness_small, lightness_big):
        path = '/content/google_images/google_images/' + str(i) + '/'
        for f in os.listdir(path):
            shutil.copyfile(path + f, new_directory + f)

move_to_group(0, 1, 1)
move_to_group(4, 35, 2)
move_to_group(35, 64, 3)

In [0]:
#Create Training and Test Set 

import os
import numpy as np
import shutil
import random

# # Creating Train / Val / Test folders (One time use)
root_dir = '/content/google_images/data/google_image_cnn/'
classes_dir = ['class_1', 'class_2', 'class_3']

#val_ratio = 0.15
test_ratio = 0.3

for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls)
    #os.makedirs(root_dir +'/val' + cls)
    os.makedirs(root_dir +'test/' + cls)


    # Creating partitions of the data after shuffling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    #val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    #print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'train/' + cls)

    #for name in val_FileNames:
        #shutil.copy(name, root_dir +'/val' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_dir +'test/' + cls)

In [0]:
# get image features

model_old = VGG16(weights='imagenet', include_top=False)


def get_input_feature(img_path):
    # img = image.load_img(img_path, target_size=(400, 400))
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model_old.predict(x)
    return features[0]


# train
all_figures = []
trainLabels = []

# need upsampling because of the unbalance of the training classes
path_1 = '/content/google_images/data/google_image_cnn/train/class_1/'
# path_1 = 'google_image_cnn/class_1/'
class_1_files = os.listdir(path_1)
trainLabels += [[1, 0, 0]] * len(class_1_files)
all_figures += [path_1 + i for i in class_1_files]

path_2 = '/content/google_images/data/google_image_cnn/train/class_2/'
# path_2 = 'google_image_cnn/class_2/'
class_2_files = os.listdir(path_2)
trainLabels += [[0, 1, 0]] * len(class_2_files)
all_figures += [path_2 + i for i in class_2_files]

path_3 = '/content/google_images/data/google_image_cnn/train/class_1/'
# path_3 = 'google_image_cnn/class_3/'
class_3_files = os.listdir(path_3)
trainLabels += [[0, 0, 1]] * len(class_3_files)
all_figures += [path_3 + i for i in class_3_files]


# a = get_input_feature(all_figures[0])
# pool = Pool(10)
# trainData = pool.map(get_input_feature, all_figures)

trainData = []
t1 = time.time()
for idx, i in enumerate(all_figures):
    a = get_input_feature(i)
    trainData.append(a)
    if idx % 1000 == 0:
        t2 = time.time()
        print(idx)
        print(t2 - t1)
        t1 = time.time()


x_all = np.asarray(trainData)
y_all = np.asarray(trainLabels)

In [0]:
# test
all_figures = []
testLabels = []

path_1 = '/content/google_images/data/google_image_cnn/test/class_1/'
# path_1 = 'google_image_cnn/class_1/'
class_1_files = os.listdir(path_1)
testLabels += [[1, 0, 0]] * len(class_1_files)
all_figures += [path_1 + i for i in class_1_files]

path_2 = '/content/google_images/data/google_image_cnn/test/class_2/'
# path_2 = 'google_image_cnn/class_2/'
class_2_files = os.listdir(path_2)
testLabels += [[0, 1, 0]] * len(class_2_files)
all_figures += [path_2 + i for i in class_2_files]

path_3 = '/content/google_images/data/google_image_cnn/train/class_3/'
# path_3 = 'google_image_cnn/class_3/'
class_3_files = os.listdir(path_3)
testLabels += [[0, 0, 1]] * len(class_3_files)
all_figures += [path_3 + i for i in class_3_files]


# a = get_input_feature(all_figures[0])
# pool = Pool(10)
# testData = pool.map(get_input_feature, all_figures)

testData = []
t1 = time.time()
for idx, i in enumerate(all_figures):
    a = get_input_feature(i)
    testData.append(a)
    if idx % 1000 == 0:
        t2 = time.time()
        print(idx)
        print(t2 - t1)
        t1 = time.time()


x_all_test = np.asarray(testData)
y_all_test = np.asarray(testLabels)

# np.savez('google_image_feature.npz', x_all=x_all, y_all=y_all)
np.savez('/content/drive/My Drive/Thesis/daytime_analysis/google_image_feature_upsampling.npz', x_all=x_all, y_all=y_all, 
         x_all_test=x_all_test, y_all_test=y_all_test)


In [0]:
# npzfile = np.load('google_image_feature_upsampling.npz')
# print npzfile.files
# x_all = npzfile['x_all']
# y_all = npzfile['y_all']

x_train = x_all
x_test = x_all_test
y_train = y_all
y_test = y_all_test


In [0]:
# the model configuration: 
# https://github.com/nealjean/predicting-poverty/blob/master/model/predicting_poverty_deploy.prototxt
model = Sequential()
model.add(Convolution2D(4096, 6, 6, activation='relu', input_shape=(12, 12, 512), subsample=(6, 6), name='input'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, 1, 1, activation='relu', subsample=(1, 1), name='conv_7'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, 1, 1, subsample=(1, 1), name='conv_8'))
model.add(AveragePooling2D((2, 2), strides=(1, 1), name='add_pool'))

model.add(Flatten(name='flatten'))
model.add(Dense(3))
model.add(Activation("softmax"))

opt = SGD(lr=1e-2)
# model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


model.fit(x_train, y_train, batch_size=100, nb_epoch=10, verbose=1)

score = model.evaluate(x_test, y_test, verbose=0)  # 0.778 after 4 epoch
print(score)


In [0]:
# get features
npzfile = np.load('/content/drive/My Drive/Thesis/nightlight.npz')
print npzfile.files
top_left_x_coords = npzfile['top_left_x_coords']
top_left_y_coords = npzfile['top_left_y_coords']
bands_data = npzfile['bands_data']


def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx


model_select = Model(input=model.input, output=model.get_layer('add_pool').output)


images_name = {} 
for i in range(64):
    if i == 0: 
        dir_ = '/content/google_images/google_images/' + str(i) + '/'
    else:
        dir_ = '/content/google_images/google_images/' + str(i+3) + '/'
    image_files = os.listdir(dir_)
    for f in image_files:
        images_name[f] = i


def get_input_feature_2(img_path):
    # img = image.load_img(img_path, target_size=(400, 400))
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model_old.predict(x)
    pool_features = model_select.predict(features)
    return pool_features[0]

def get_daytime_feature(sample):
    idx, wealth, x, y = sample
    print idx
    lon_idx, lat_idx = get_cell_idx(x, y, top_left_x_coords, top_left_y_coords)
    left_idx = lon_idx - 5
    right_idx = lon_idx + 4
    up_idx = lat_idx - 5
    low_idx = lat_idx + 4
    features_100 = []
    for i in xrange(left_idx, right_idx + 1):
        for j in xrange(up_idx, low_idx + 1):
            file_name = str(i) + '_' + str(j) + '.jpg'
            if file_name in images_name:
                luminosity = images_name[file_name]
                feature = get_input_feature_2('/content/google_images/google_images/' + str(luminosity) + '/' + file_name)
                features_100.append(feature)
    if len(features_100) == 0:
        print('nononono: ' + str(idx))
        return np.asarray([np.nan] * 4096 + [wealth]).tolist()
    else:
        features_100 = np.asarray(features_100)
        return np.append(np.mean(features_100, axis=0), wealth).tolist()


clusters = pd.read_csv('/content/drive/My Drive/Thesis/DHS_Data/bangladesh_cluster_avg_asset_2013_updated_new.csv')
clusters['feature'] = clusters.apply(lambda x: get_daytime_feature([x['cluster'], x['wlthindf'], 
                                                                    x['longitude'], x['latitude']]), axis=1)

data_all = clusters['feature']
data_all = np.asarray([i for i in data_all])
data_all = data_all[~np.isnan(data_all).any(axis=1)]

np.savetxt('/content/drive/My Drive/Thesis/daytime_analysis/google_image_features_cnn_retrain.csv', data_all)


In [0]:
data_all = np.loadtxt('/content/drive/My Drive/Thesis/daytime_analysis/google_image_features_cnn_retrain.csv')

alphas_list = np.logspace(-1, 5, 7)

final = []
for alpha in alphas_list:
    kf = KFold(n_splits=10, shuffle=True)
    scores = []
    for train_index, test_index in kf.split(data_all):
        reg = Ridge(alpha=alpha)
        train = data_all[train_index]
        test = data_all[test_index]
        train_x = train[:, :-1]
        train_y = train[:, -1]
        test_x = test[:, :-1]
        test_y = test[:, -1]
        # reduce dimensions to avoid overfitting
        pca = PCA(n_components=100)
        pca.fit(train_x)
        train_x = pca.transform(train_x)
        test_x = pca.transform(test_x)
        reg.fit(train_x, train_y)
        s = reg.score(test_x, test_y)
        scores.append(s)
    final.append(np.mean(scores))

print 'R^2 of the best model: {:.3f}'.format(np.max(final))
